In [1]:
# Chinese Remainder Theorem.
def chineseRemainderTheorem(arguments):    
    """Chinese Remainder Theorem.
    Input is list of congruences (RHS a, modulo m). Returns solution mod m_1*m_2...m_n."""
    # Check if moduli are pairwise relatively prime.
    for i in range(len(arguments)-1):
        for j in range(i+1, len(arguments)):
            if gcd(arguments[i][1], arguments[j][1]) != 1:
                print("Moduli must be pairwise coprime.")
                return False
    
    # Compute intermediate values.
    M = reduce(lambda x,y: x*y, [each[1] for each in arguments])
    triplets = [] # [..,(a, b, c),..]
    skip = 0
    
    for iterations in range(len(arguments)):
        M_val = 1
        for i in range(len(arguments)):
            if i != skip:
                M_val *= arguments[i][1]

        triplets.append((arguments[skip][0], M_val, int(Integers(arguments[skip][1])(M_val)^-1)))
        skip += 1
        
    return Mod(reduce(lambda x,y: x+y, [each[0]*each[1]*each[2] for each in triplets]), M)

# Test suite.
print(chineseRemainderTheorem([(1, 2), (2, 3), (4, 5), (0, 7)]))

119


In [2]:
# Fermat's Factorization Method.
def fermatFactorization(n):
    # Returns:
    # --None, if neither odd nor a multiple of 4,
    # --(x, y), s.t. n^2 = x^2 - y^2.
    
    if Mod(n, 2) != 1 and Mod(n, 4) != 0:
        return None 
    x0 = ceil(sqrt(n))
    while True:
        diff = (x0)^2 - n
        if is_square(diff):
            return (x0, sqrt(diff))
        x0 += 1
        
# Test suite.
print(fermatFactorization(901))

(35, 18)


In [3]:
# Basic Factorization Principle algorithm.
def basicFactorizationPrinciple(x, y, n):
    # Returns:
    # --None, if x^2 \neq y^2 or x = +\-y (mod n),
    # --gcd(x-y, n) - a nontrivial factor of n!
    if Mod(x^2, n) != Mod(y^2, n):
        print("Error: " + str(x) + "^2 isn't congruent to " + str(y) + "^2 (mod " + str(n) + ").")
        return None
    elif Mod(x, n) == Mod(y, n):
        print("Error: " + str(x) + " is congruent to " + str(y) + " modulo" + str(n) + ".")
        return None
    elif Mod(x, n) == Mod(-y, n):
        print("Error: " + str(x) + " is congruent to -" + str(y) + " modulo" + str(n) + ".")
        return None
    return gcd(x-y, n)

# Test suite.
x = (2*516107)
y = 187722
n = 642401
print(basicFactorizationPrinciple(x, y, n))

1129


In [4]:
# Strong Fermat factorization algorithm.
def strongFermat(n, b=2):
    # Returns:
    # --None, if n is even (n must be odd to factor out powers of 2),
    # --True/False
    # Prints list of b_i values.
    # Default is b=2 unless specified otherwise.
    if Mod(n, 2) == 0:
        print("Error: n must be ODD to use strong Fermat test.")
        return None
    
    # Extract k, s by factoring n-1 (where n-1 = 2^k * s)
    k = 0
    s = n-1
    while gcd(s, 2) > 1:
        k += 1
        s = s/2
    
    betas = []
    j = k # j is the SMALLEST index for which b_j = 1 (mod n)
    # Store value of b_i for each iteration in betas
    for i in range(k+1):
        if i == 0:
            betas.append(Mod(b^s, n)) # b_0 = b^s (mod n)
        else:
            betas.append(Mod( (betas[i-1])^2 , n )) # b_i = (b^{i-1})^2 (mod n)
        if betas[i] == 1:
            j = min(i, j)
    print("*b values* " + str(betas))

    # Returns True if b_k = 1 (mod n) AND (j = 0 or b_{j-1} = -1 (mod n))
    # Without feedback: return betas[k] == 1 and (j == 0 or betas[j-1] == -1)
    if betas[k] != 1:
        print("Failed: b_k equals " + str(betas[k] + " rather than 1 (mod n)."))
        return False
    
    if j == 0:
        print("Passed: j is 0. (Note: j is smallest index for which b_j = 1 (mod n).)")
        return True
    elif betas[j-1] == -1:
        print("Passed: b_{j-1} equals -1. (Note: j is smallest index for which b_j = 1 (mod n).)")
        return True
    else:
        print("Failed: b_j equals 1 (mod n), BUT j \neq 0 and b_{j-1} \neq -1 (mod n).")
        return False
    
# Test suite.
print(strongFermat(2047))

*b values* [1, 1]
Passed: j is 0. (Note: j is smallest index for which b_j = 1 (mod n).)
True


In [1]:
# Algorithm to detect if n is a Carmichael number.
def isCarmichael(n):
# Must be squarefree, and (p-1)|(n-1) for all prime factors p (of n)
    if n <= 2 or is_prime(n):
        print("Error: n must be COMPOSITE, and greater than 2.")
        return False
    primes = list(factor(n))
    print("*Prime Factors* " + str(primes))
    # If not squarefree, return False.
    # Verify if the weak Fermat test holds, mod each prime factor.
    for each in primes:
        if each[1] != 1 or Mod(n-1, each[0]-1) != 0:
            return False
    return True

# Test suite.
print(isCarmichael(1729))

In [5]:
# Euclid and LCM algorithms.

def euclid(a, b):
# Computes the gcd via the Euclidean algorithm.
    if b == 0:
        return a
    return euclid(b, Mod(a, b))

def pairwiseLCM(x, y):
    return Integer(x*y) // Integer(euclid(x, y))

def recursiveLCM(numbers):
    if len(numbers) == 2:
        return Integer(numbers[0] * numbers[1]) // Integer(euclid(numbers[0], numbers[1]))
    return pairwiseLCM(numbers[0], recursiveLCM(numbers[1:]))